In [19]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

os.environ['environment'] = 'interactive'
os.environ['UTILS_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/python-utils-ak'
os.environ['PROJECT_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat' 
os.environ['SQLITE_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/data.sqlite' 
sys.path.append(os.environ['UTILS_PATH'])
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [77]:
import itertools
list(itertools.permutations(range(5), 3))

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 1),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 4),
 (0, 4, 1),
 (0, 4, 2),
 (0, 4, 3),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 2, 0),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 0),
 (1, 3, 2),
 (1, 3, 4),
 (1, 4, 0),
 (1, 4, 2),
 (1, 4, 3),
 (2, 0, 1),
 (2, 0, 3),
 (2, 0, 4),
 (2, 1, 0),
 (2, 1, 3),
 (2, 1, 4),
 (2, 3, 0),
 (2, 3, 1),
 (2, 3, 4),
 (2, 4, 0),
 (2, 4, 1),
 (2, 4, 3),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 4),
 (3, 1, 0),
 (3, 1, 2),
 (3, 1, 4),
 (3, 2, 0),
 (3, 2, 1),
 (3, 2, 4),
 (3, 4, 0),
 (3, 4, 1),
 (3, 4, 2),
 (4, 0, 1),
 (4, 0, 2),
 (4, 0, 3),
 (4, 1, 0),
 (4, 1, 2),
 (4, 1, 3),
 (4, 2, 0),
 (4, 2, 1),
 (4, 2, 3),
 (4, 3, 0),
 (4, 3, 1),
 (4, 3, 2)]

In [78]:
numpy.random.permutation(range(5))

array([2, 4, 3, 0, 1])

In [85]:
sku_ids = range(62, 82)
boiling_model_ids = range(9, 16)
n = 5

In [86]:
skus = [cast_model(RicottaSKU, sku_id) for sku_id in sku_ids]
skus

[<RicottaSKU 62>,
 <RicottaSKU 63>,
 <RicottaSKU 64>,
 <RicottaSKU 65>,
 <RicottaSKU 66>,
 <RicottaSKU 67>,
 <RicottaSKU 68>,
 <RicottaSKU 69>,
 <RicottaSKU 70>,
 <RicottaSKU 71>,
 <RicottaSKU 72>,
 <RicottaSKU 73>,
 <RicottaSKU 74>,
 <RicottaSKU 75>,
 <RicottaSKU 76>,
 <RicottaSKU 77>,
 <RicottaSKU 78>,
 <RicottaSKU 79>,
 <RicottaSKU 80>,
 <RicottaSKU 81>]

In [101]:
values = []
for i in range(n):
    boiling_skus = []
    boiling_model = cast_model(RicottaBoiling, random.choice(boiling_model_ids))
    
    n_boilings = random.choice([2, 3])
    for _ in range(n_boilings):
        boiling_skus.append(random.choice([sku for sku in skus if boiling_model in sku.made_from_boilings]))
    
    boiling_skus = list(sorted(boiling_skus, key=lambda sku: sku.name))
    for sku in boiling_skus:
        values.append([i, sku, 1])
values

[[0, <RicottaSKU 63>, 1],
 [0, <RicottaSKU 64>, 1],
 [1, <RicottaSKU 67>, 1],
 [1, <RicottaSKU 66>, 1],
 [1, <RicottaSKU 79>, 1],
 [2, <RicottaSKU 80>, 1],
 [2, <RicottaSKU 80>, 1],
 [3, <RicottaSKU 80>, 1],
 [3, <RicottaSKU 80>, 1],
 [4, <RicottaSKU 66>, 1],
 [4, <RicottaSKU 79>, 1],
 [4, <RicottaSKU 79>, 1]]

In [102]:
df = pd.DataFrame(values, columns=['boiling_id', 'sku', 'volume'])
df

,boiling_id,sku,volume
0,0,<RicottaSKU 63>,1
1,0,<RicottaSKU 64>,1
2,1,<RicottaSKU 67>,1
3,1,<RicottaSKU 66>,1
4,1,<RicottaSKU 79>,1
5,2,<RicottaSKU 80>,1
6,2,<RicottaSKU 80>,1
7,3,<RicottaSKU 80>,1
8,3,<RicottaSKU 80>,1
9,4,<RicottaSKU 66>,1


In [ ]:
boiling_model_ids = range(9, 16)